Inversiones
===

Notas de clase sobre ingeniería economica avanzada usando Python

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia  

**Software utilizado**
>Este es un documento interactivo escrito como un notebook de [Jupyter](http://jupyter.org/) , en el cual se presenta un tutorial sobre finanzas corporativas usando Python. Los notebooks de Jupyter permiten incoporar simultáneamente código, texto, gráficos y ecuaciones. El código presentado en este notebook puede ejecutarse en los sistemas operativos Linux y OS X.  

>Haga click [aquí](https://github.com/jdvelasq/guias-de-instalacion) para obtener instrucciones detalladas sobre como instalar Jupyter en Windows y Mac OS X.

>Descargue la última versión de este documento a su disco duro; luego, carguelo y ejecutelo en línea en Try [Jupyter!](https://try.jupyter.org/)



#### Contenido

>

**Bibliografía**

> * [1] SAS/ETS 14.1 User's Guide, 2015.   
* [2] **hp** 12c platinum financial calculator. User's guide. 
* [3] HP Business Consultant II Owner's manual.
* [4] C.S. Park and G.P. Sharp-Bette. Advanced Engineering Economics. John Wiley & Sons, Inc., 1990.

In [2]:
import cashflows as cf

# Depreciación

[Contenido](#Contenido)

Permite la deducción de los costos de las inversiones (pérdida de valor de ellas) con el fin de reducir el impuesto de renta.

## Depreciación en linea recta

[Contenido](#Contenido)

La depreciación en todos los años es igual.

* $P$ -- valor de compra en libros.

* $N$ -- número de años en que ocurre la depreciación (vida útil del activo).

* $D_n$ -- depreciación en el año $n$.

$$D_n = \frac{P-F}{N}$$

* $F$ -- valor de salvamento al final de la vida útil.

**Ejemplo.--** Un activo cuesta  \$ 200 . Si la depreciación ocurre durante 5 años, cuál es su depreciación anual?.

In [68]:
#
# los datos del activo se ingresan como una dupla 
# conformada asi: (costo, vida util)

costs = cf.cashflow(const_value=0, nper=5, spec=(0, 200)) # Costo del
life = cf.cashflow(const_value=0, nper=5, spec=(0, 5))
#
# 'SL' indica la depreciación en linea recta
#                        (cost, life)
cf.depreciation_sl(costs=costs, life=life,noprint=False)

#
# el resultado se imprime en forma de tabla:
#
#  * Beg. Book Value -- representa el valor total de los activos al inicio del periodo
#  * Cost -- Costo en activos en el periodo
#  * Depre -- depreciación total para el periodo t
#  * Accum. Depre --- depreciación acumulada por periodo
#  * End.Book Value -- valor final de los activos al final del periodo

t       Beg.    Cost  Depre.  Accum.    End.
        Book                  Depre.    Book
       Value                           Value
--------------------------------------------
(0,)    0.00  200.00    0.00    0.00  200.00
(1,)  200.00    0.00   40.00   40.00  160.00
(2,)  160.00    0.00   40.00   80.00  120.00
(3,)  120.00    0.00   40.00  120.00   80.00
(4,)   80.00    0.00   40.00  160.00   40.00


## Depreciación por suma de los dígitos de los años (o depreciación acelerada)

[Contenido](#Contenido)

En este método de depreciación se hace mayor al principio y más pequeña al final. Se basa en la progresión desde $N$ hasta 1.

$$S = \sum_{i=1}^N i = \frac{1}{2} N(N + 1)$$

$$D_n = (P - F) * \frac{(N - n + 1)}{S} = 2 * (P - F) * \frac{N-n+1}{N(N+1)}$$ 

**Ejemplo.--** Calcule la depreciación por el método de la suma de los dígitos de los años para el activo del ejemplo anterior.

In [70]:
## 'SOYD' -- sum of year's digits
cf.depreciation_soyd(costs=costs, life=life,noprint=False) # (cost, life)
# noprint es para imprimir la tabla cuando se indica False


t       Beg.    Cost  Depre.  Accum.    End.
        Book                  Depre.    Book
       Value                           Value
--------------------------------------------
(0,)    0.00  200.00    0.00    0.00  200.00
(1,)  200.00    0.00   66.67   66.67  133.33
(2,)  133.33    0.00   53.33  120.00   80.00
(3,)   80.00    0.00   40.00  160.00   40.00
(4,)   40.00    0.00   26.67  186.67   13.33


## Modelado de la depreciación usando `cashflow`

In [74]:
##
## es posible obtener la columna 'depreciation' como un flujo de efectivo teniendo en cuenta los periodos anuales y
## en cuales periodos se reflejaran los costos 
##
costs = cf.cashflow(const_value=0, 
                    nper=16, 
                    spec=[(0, 1000), (8, 1000)], 
                    pyr=4)
life = cf.cashflow(const_value=0, 
                   nper=16, 
                   spec=[(0, 4), (8, 4)], 
                   pyr=4)
delay22 = cf.cashflow(const_value=0, 
                      nper=16, 
                      spec=[(0, 2), (8, 2)],
                      pyr=4)

cf.depreciation_sl(costs=costs,
                   life=life,
                   delay=delay22)

    Qtr0   Qtr1   Qtr2   Qtr3
0   0.00   0.00   0.00 250.00
1 250.00 250.00 250.00   0.00
2   0.00   0.00   0.00 250.00
3 250.00 250.00 250.00   0.00

In [75]:
##
## es posible obtener la tabla de depreciación
##
cf.depreciation_sl(costs=costs, life=life, delay=delay22, noprint=False)

t         Beg.    Cost  Depre.  Accum.    End.
          Book                  Depre.    Book
         Value                           Value
----------------------------------------------
(0, 0)    0.00 1000.00    0.00    0.00 1000.00
(0, 1) 1000.00    0.00    0.00    0.00 1000.00
(0, 2) 1000.00    0.00    0.00    0.00 1000.00
(0, 3) 1000.00    0.00  250.00  250.00  750.00
(1, 0)  750.00    0.00  250.00  500.00  500.00
(1, 1)  500.00    0.00  250.00  750.00  250.00
(1, 2)  250.00    0.00  250.00 1000.00    0.00
(1, 3)    0.00    0.00    0.00 1000.00    0.00
(2, 0)    0.00 1000.00    0.00 1000.00 1000.00
(2, 1) 1000.00    0.00    0.00 1000.00 1000.00
(2, 2) 1000.00    0.00    0.00 1000.00 1000.00
(2, 3) 1000.00    0.00  250.00 1250.00  750.00
(3, 0)  750.00    0.00  250.00 1500.00  500.00
(3, 1)  500.00    0.00  250.00 1750.00  250.00
(3, 2)  250.00    0.00  250.00 2000.00    0.00
(3, 3)    0.00    0.00    0.00 2000.00    0.00


## Efecto de la depreciación sobre el impuesto de renta y el flujo de caja

[Contenido](#Contenido)

**Caso 1: sin depreciación.**

In [91]:
# crea el flujo de caja
i = cf.cashflow(const_value=[500]*5)          # ingresos brutos: $ 500 por periodo
g = cf.cashflow(const_value=[250]*5)                         # gastos operativos: $ 250 por periodo
x = i-g                                      # ingreso antes de impuestos
x

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)-(4,) [5] 250.00 

In [92]:
x = x + cf.after_tax_cashflow(x, tax_rate = cf.nominal_rate(const_value=[35] * 5)) # impuesto de renta del 35%
x                                              # ingreso despues de impuestos

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)-(4,) [5] 337.50 

**Caso 2: depreciación en linea recta.**

In [120]:
## considere un activo depreciable
d = cf.depreciation_sl(costs=cf.cashflow(const_value=0, nper=5, spec=(0, 200)), 
                       life=cf.cashflow(const_value=0, nper=5, spec=(0, 5)),
                       noprint=True)
d

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)           0.00 
       (1,)-(4,) [4] 40.00 

In [121]:
# crea el flujo de caja
i = cf.cashflow(const_value=[500]*5)          # ingresos brutos: $ 500 por periodo
g = cf.cashflow(const_value=[250]*5)                         # gastos operativos: $ 250 por periodo
x = i-g                                      # ingreso antes de impuestos
x

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)-(4,) [5] 250.00 

In [122]:
x = x + d                     # depreciacion
x                            # ingreso antes de impuestos

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)          250.00 
       (1,)-(4,) [4] 290.00 

In [123]:
x = x + cf.after_tax_cashflow(x, tax_rate = cf.nominal_rate(const_value=[35] * 5)) # impuesto de renta del 35%
x                                              # ingreso despues de impuestos

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)          337.50 
       (1,)-(4,) [4] 391.50 

In [124]:
x - d  # flujo de caja

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)          337.50 
       (1,)-(4,) [4] 351.50 

Note que en el resultado anterior el ingreso después de impuestos es mayor que en el caso 1.

**Caso 2: depreciación acelerada.**

In [130]:
## activo depreciable
costs=cf.cashflow(const_value=0, nper=5, spec=(0, 200))
life=cf.cashflow(const_value=0, nper=5, spec=(0, 5))
d =cf.depreciation_soyd(costs=costs, life=life)
d

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)    0.00 
       (1,)   66.67 
       (2,)   53.33 
       (3,)   40.00 
       (4,)   26.67 

In [131]:
# crea el flujo de caja
i = cf.cashflow(const_value=[500]*5)          # ingresos brutos: $ 500 por periodo
g = cf.cashflow(const_value=[250]*5)                         # gastos operativos: $ 250 por periodo
x = i-g                                      # ingreso antes de impuestos
x

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)-(4,) [5] 250.00 

In [132]:
x = x + d                     # depreciacion
x                             # ingreso antes de impuestos

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)   250.00 
       (1,)   316.67 
       (2,)   303.33 
       (3,)   290.00 
       (4,)   276.67 

In [133]:
x = x + cf.after_tax_cashflow(x, tax_rate = cf.nominal_rate(const_value=[35] * 5)) # impuesto de renta del 35%
x                                              # ingreso despues de impuestos

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)   337.50 
       (1,)   427.50 
       (2,)   409.50 
       (3,)   391.50 
       (4,)   373.50 

In [134]:
x - d  # flujo de caja

Time Series:
Start = (0,)
End = (4,)
pyr = 1
Data = (0,)   337.50 
       (1,)   360.83 
       (2,)   356.17 
       (3,)   351.50 
       (4,)   346.83 

# Cuentas de Ahorros

[Contenido](#Contenido)

**Ejercicio.--** Construya el balance para 12 meses, si habre una cuenta de ahorro que paga el 2% mensual, y hace 12 depósitos de \$ 100 (puede usar Microsoft Excel).

**Ejercicio.--** Para el ejercicio anterior, cuánto es la suma de los intereses aritmética de los intereses recibidos?

**Ejercicio.--** Construya el balance para 12 meses, si usted tiene una cuenta de ahorro que paga el 2% mensual con un capital inicial de \$ 1000, y hace retiros mensuales de \$ 100 iniciando hoy (puede usar Microsoft Excel).

**Ejercicio.--** Para el ejercicio anterior, cuánto es la suma de los intereses aritmética de los intereses recibidos?

# Modelado de cuentas de ahorro usando cashflow

[Contenido](#Contenido)

**`cashflows`** permite el modelado de cuentas de ahorro (e intrumentos financieros similares).

**Ejemplo.--** Construya el balance para 12 meses, si habre una cuenta de ahorro que paga el 2% mensual, y hace 13 depósitos de \$ 100 (con el primer depósito se abre la cuenta).

In [153]:
cflo = cf.cashflow(const_value=[100] * 13)
nrate = cf.nominal_rate([2] * 13)
x = cf.savings(deposits = cflo,       # deposito periodico     
               initbal  =    0,      # balance inicial
               rate     =    nrate,   # tasa de interes mensual 
               noprint=False)      

t     Beginning   Deposit    Earned    Ending
        Balance            Interest   Balance
----------------------------------------------
(0,)       0.00    100.00      0.00    100.00
(1,)     100.00    100.00      2.00    202.00
(2,)     202.00    100.00      4.04    306.04
(3,)     306.04    100.00      6.12    412.16
(4,)     412.16    100.00      8.24    520.40
(5,)     520.40    100.00     10.41    630.81
(6,)     630.81    100.00     12.62    743.43
(7,)     743.43    100.00     14.87    858.30
(8,)     858.30    100.00     17.17    975.46
(9,)     975.46    100.00     19.51   1094.97
(10,)   1094.97    100.00     21.90   1216.87
(11,)   1216.87    100.00     24.34   1341.21
(12,)   1341.21    100.00     26.82   1468.03


In [154]:
cflo = cf.cashflow(const_value=[100] * 13)
nrate = cf.nominal_rate([2] * 13)
x = cf.savings(deposits = cflo,       # deposito periodico     
               initbal  =    0,      # balance inicial
               rate     =    nrate)   # tasa de interes mensual 
 
## intereses como lista 
x

(Time Series:
 Start = (0,)
 End = (12,)
 pyr = 1
 Data = (0,)     0.00 
        (1,)     2.00 
        (2,)     4.04 
        (3,)     6.12 
        (4,)     8.24 
        (5,)    10.41 
        (6,)    12.62 
        (7,)    14.87 
        (8,)    17.17 
        (9,)    19.51 
        (10,)   21.90 
        (11,)   24.34 
        (12,)   26.82 , Time Series:
 Start = (0,)
 End = (12,)
 pyr = 1
 Data = (0,)     100.00 
        (1,)     202.00 
        (2,)     306.04 
        (3,)     412.16 
        (4,)     520.40 
        (5,)     630.81 
        (6,)     743.43 
        (7,)     858.30 
        (8,)     975.46 
        (9,)    1094.97 
        (10,)   1216.87 
        (11,)   1341.21 
        (12,)   1468.03 )

**Ejemplo.--** Realice el mismo ejemplo anterior, pero considerando que la tasa de interes es del 4% a partir de $t$ = 5.

In [155]:
nrate= cf.nominal_rate(const_value=[2] * 3,spec=(5,4))
cflo = cf.cashflow(const_value=[100] * 12)
x = cf.savings(deposits = cflo,       # deposito periodico     
               initbal  =    0,      # balance inicial
               rate     =    nrate, # tasa de interes mensual
               noprint = False)    


t     Beginning   Deposit    Earned    Ending
        Balance            Interest   Balance
----------------------------------------------
(0,)       0.00    100.00      0.00    100.00
(1,)     100.00    100.00      2.00    202.00
(2,)     202.00    100.00      4.04    306.04
(3,)     306.04    100.00      6.12    412.16
(4,)     412.16    100.00      8.24    520.40
(5,)     520.40    100.00     20.82    641.22
(6,)     641.22    100.00     25.65    766.87
(7,)     766.87    100.00     30.67    897.54
(8,)     897.54    100.00     35.90   1033.45
(9,)    1033.45    100.00     41.34   1174.78
(10,)   1174.78    100.00     46.99   1321.77
(11,)   1321.77    100.00     52.87   1474.65
(12,)   1474.65    100.00     58.99   1633.63


**Ejemplo.--** Se tiene una cuenta con un saldo inicial de \$ 100. Se hacen depósitos  al final de cada trimestre por \$ 50 (se hará el primer depósito en 3 meses). La tasa de interés es del 2% mensual y cambiará a 1% a partir del 13avo mes (incluido). Cuál será el saldo al final de mes 24?. 

In [152]:
nrate= cf.nominal_rate(const_value=[2] * 25,spec=(13,1))
cflo = cf.cashflow(const_value=0, nper=25, spec=[(t*3,50) for t in range(1,9)] )

x = cf.savings(deposits = cflo,       # deposito periodico     
               initbal  = 100,      # balance inicial
               rate     = nrate, # tasa de interes mensual
               noprint = False) 



t     Beginning   Deposit    Earned    Ending
        Balance            Interest   Balance
----------------------------------------------
(0,)     100.00      0.00      2.00    102.00
(1,)     102.00      0.00      2.04    104.04
(2,)     104.04      0.00      2.08    106.12
(3,)     106.12     50.00      2.12    158.24
(4,)     158.24      0.00      3.16    161.41
(5,)     161.41      0.00      3.23    164.64
(6,)     164.64     50.00      3.29    217.93
(7,)     217.93      0.00      4.36    222.29
(8,)     222.29      0.00      4.45    226.73
(9,)     226.73     50.00      4.53    281.27
(10,)    281.27      0.00      5.63    286.89
(11,)    286.89      0.00      5.74    292.63
(12,)    292.63     50.00      5.85    348.48
(13,)    348.48      0.00      3.48    351.97
(14,)    351.97      0.00      3.52    355.49
(15,)    355.49     50.00      3.55    409.04
(16,)    409.04      0.00      4.09    413.13
(17,)    413.13      0.00      4.13    417.26
(18,)    417.26     50.00      4.

# Préstamos

## Bullet Loan

Este es un tipo de préstamo en que se pagan únicamente intereses durante la vida del crédito; con la última cuota se paga el capital.

En el siguiente ejemplo se usa **`cashflow`** para modelar un préstamo de \$ 1000 a una tasa de interés del 10% y una duración de 10 periodos de tiempo.

In [159]:
nrate = cf.nominal_rate(const_value=10, nper=11)
x=cf.bullet_loan(amount=1000, nrate=nrate, dispoints=0, orgpoints=0, prepmt=None)  # doctest: +NORMALIZE_WHITESPACE
x


t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(2,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(3,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(4,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(5,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(6,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(7,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(8,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(9,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(10,) 1000.00   10.00 1100.00  100.00 1000.00    0.00

In [160]:
##
## todos los tipos de créditos tiene funciones 
## para obtener las columnas de la tabla de 
## amortizacion como un flujo de efectivo
##
x.intpmt

Time Series:
Start = (0,)
End = (10,)
pyr = 1
Data = (0,)              0.00 
       (1,)-(10,) [10] 100.00 

In [161]:
sum(x.intpmt)

1000.0

In [163]:
x.ppalpmt  ## pagos a capital

Time Series:
Start = (0,)
End = (10,)
pyr = 1
Data = (0,)-(9,) [10]    0.00 
       (9,)           1000.00 

In [164]:
x.begppalbal ## balance inicial del periodo

Time Series:
Start = (0,)
End = (10,)
pyr = 1
Data = (0,)               0.00 
       (1,)-(10,) [10] 1000.00 

In [165]:
x.endppalbal ## balance final del periodo

Time Series:
Start = (0,)
End = (10,)
pyr = 1
Data = (0,)-(9,) [10] 1000.00 
       (9,)              0.00 

In [168]:
##
## se admiten el prepago de la deuda sin penalizacion
## a continuación se modela un prepago de deuda de $ 500 en t = 5
##
nrate = cf.nominal_rate(const_value=10, nper=11)
x=cf.bullet_loan(amount=1000,
                 nrate=nrate, 
                 dispoints=0, 
                 orgpoints=0, 
                 prepmt=cf.cashflow(const_value=0, nper=11,spec=(5,500))) 
x


t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(2,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(3,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(4,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(5,)  1000.00   10.00  600.00  100.00  500.00  500.00
(6,)   500.00   10.00   50.00   50.00    0.00  500.00
(7,)   500.00   10.00   50.00   50.00    0.00  500.00
(8,)   500.00   10.00   50.00   50.00    0.00  500.00
(9,)   500.00   10.00   50.00   50.00    0.00  500.00
(10,)  500.00   10.00  550.00   50.00  500.00    0.00

In [170]:
##
## Si el prepago de deuda es superior al monto, 
## se ajusta al saldo
##
nrate = cf.nominal_rate(const_value=10, nper=11)
x=cf.bullet_loan(amount=1000,
                 nrate=nrate, 
                 dispoints=0, 
                 orgpoints=0, 
                 prepmt=cf.cashflow(const_value=0, nper=11,spec=(5,5000))) 
x

t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(2,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(3,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(4,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(5,)  1000.00   10.00 1100.00  100.00 1000.00    0.00
(6,)     0.00   10.00    0.00    0.00    0.00    0.00
(7,)     0.00   10.00    0.00    0.00    0.00    0.00
(8,)     0.00   10.00    0.00    0.00    0.00    0.00
(9,)     0.00   10.00    0.00    0.00    0.00    0.00
(10,)    0.00   10.00    0.00    0.00    0.00    0.00

In [174]:
##
## este tipo de crédito admite una tasa de interés variable
##
nrate = cf.nominal_rate(const_value=10, nper=11, spec=(6,8)) # cambia en el periodo 6
x=cf.bullet_loan(amount=1000,
                 nrate=nrate, 
                 dispoints=0, 
                 orgpoints=0, 
                 prepmt=None) 
x

t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(2,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(3,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(4,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(5,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(6,)  1000.00    8.00   80.00   80.00    0.00 1000.00
(7,)  1000.00    8.00   80.00   80.00    0.00 1000.00
(8,)  1000.00    8.00   80.00   80.00    0.00 1000.00
(9,)  1000.00    8.00   80.00   80.00    0.00 1000.00
(10,) 1000.00    8.00 1080.00   80.00 1000.00    0.00

## Fixed loan

In [175]:
##
## Este tipo es similar a las anteriores, pero siempre 
## se abona la misma cantidad a capital. El pago periodico
## es igual al abono a capital más el interés sobre saldos.
##
nrate = cf.nominal_rate(const_value=10, nper=11)
cf.fixed_ppal_loan(amount=1000, 
                   nrate=nrate, 
                   grace=0, 
                   dispoints=0, 
                   orgpoints=0,
                   prepmt=None, 
                   balloonpmt=None)


t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  200.00  100.00  100.00  900.00
(2,)   900.00   10.00  190.00   90.00  100.00  800.00
(3,)   800.00   10.00  180.00   80.00  100.00  700.00
(4,)   700.00   10.00  170.00   70.00  100.00  600.00
(5,)   600.00   10.00  160.00   60.00  100.00  500.00
(6,)   500.00   10.00  150.00   50.00  100.00  400.00
(7,)   400.00   10.00  140.00   40.00  100.00  300.00
(8,)   300.00   10.00  130.00   30.00  100.00  200.00
(9,)   200.00   10.00  120.00   20.00  100.00  100.00
(10,)  100.00   10.00  110.00   10.00  100.00    0.00

In [176]:
##
## periodo de gracia de 3
##
nrate = cf.nominal_rate(const_value=10, nper=11)
cf.fixed_ppal_loan(amount=1000, 
                   nrate=nrate, 
                   grace=3, 
                   dispoints=0, 
                   orgpoints=0,
                   prepmt=None, 
                   balloonpmt=None)


t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(2,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(3,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(4,)  1000.00   10.00  242.86  100.00  142.86  857.14
(5,)   857.14   10.00  228.57   85.71  142.86  714.29
(6,)   714.29   10.00  214.29   71.43  142.86  571.43
(7,)   571.43   10.00  200.00   57.14  142.86  428.57
(8,)   428.57   10.00  185.71   42.86  142.86  285.71
(9,)   285.71   10.00  171.43   28.57  142.86  142.86
(10,)  142.86   10.00  157.14   14.29  142.86    0.00

In [177]:
##
## este tipo de crédito admite una tasa de interés variable
##
nrate = cf.nominal_rate(const_value=10, nper=11, spec=(6,8)) # cambia en el periodo 6
cf.fixed_ppal_loan(amount=1000, 
                   nrate=nrate, 
                   grace=0, 
                   dispoints=0, 
                   orgpoints=0,
                   prepmt=None, 
                   balloonpmt=None)



t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  200.00  100.00  100.00  900.00
(2,)   900.00   10.00  190.00   90.00  100.00  800.00
(3,)   800.00   10.00  180.00   80.00  100.00  700.00
(4,)   700.00   10.00  170.00   70.00  100.00  600.00
(5,)   600.00   10.00  160.00   60.00  100.00  500.00
(6,)   500.00    8.00  140.00   40.00  100.00  400.00
(7,)   400.00    8.00  132.00   32.00  100.00  300.00
(8,)   300.00    8.00  124.00   24.00  100.00  200.00
(9,)   200.00    8.00  116.00   16.00  100.00  100.00
(10,)  100.00    8.00  108.00    8.00  100.00    0.00

In [180]:
##
## este tipo de crédito admite pagos adicionales programados
## como abono a la deuda
##
nrate = cf.nominal_rate(const_value=10, nper=11)
cf.fixed_ppal_loan(amount=1000, 
                   nrate=nrate, 
                   grace=0, 
                   dispoints=0, 
                   orgpoints=0,
                   prepmt=None, 
                   balloonpmt=cf.cashflow(const_value=0, nper = 11, spec=(5, 500)))




t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  150.00  100.00   50.00  950.00
(2,)   950.00   10.00  145.00   95.00   50.00  900.00
(3,)   900.00   10.00  140.00   90.00   50.00  850.00
(4,)   850.00   10.00  135.00   85.00   50.00  800.00
(5,)   800.00   10.00  630.00   80.00  550.00  250.00
(6,)   250.00   10.00   75.00   25.00   50.00  200.00
(7,)   200.00   10.00   70.00   20.00   50.00  150.00
(8,)   150.00   10.00   65.00   15.00   50.00  100.00
(9,)   100.00   10.00   60.00   10.00   50.00   50.00
(10,)   50.00   10.00   55.00    5.00   50.00    0.00

In [181]:
##
## este tipo de crédito admite una tasa de interés variable
##
nrate = cf.nominal_rate(const_value=10, nper=11,spec=(6,8)) # cambia en el periodo 6
cf.fixed_ppal_loan(amount=1000, 
                   nrate=nrate, 
                   grace=0, 
                   dispoints=0, 
                   orgpoints=0,
                   prepmt=None, 
                   balloonpmt=cf.cashflow(const_value=0, nper = 11, spec=(5, 500)))

t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  150.00  100.00   50.00  950.00
(2,)   950.00   10.00  145.00   95.00   50.00  900.00
(3,)   900.00   10.00  140.00   90.00   50.00  850.00
(4,)   850.00   10.00  135.00   85.00   50.00  800.00
(5,)   800.00   10.00  630.00   80.00  550.00  250.00
(6,)   250.00    8.00   70.00   20.00   50.00  200.00
(7,)   200.00    8.00   66.00   16.00   50.00  150.00
(8,)   150.00    8.00   62.00   12.00   50.00  100.00
(9,)   100.00    8.00   58.00    8.00   50.00   50.00
(10,)   50.00    8.00   54.00    4.00   50.00    0.00

In [183]:
##
## ejemplo anterior con periodo de gracia
##
nrate = cf.nominal_rate(const_value=10, nper=11,spec=(6,8)) # cambia en el periodo 6
cf.fixed_ppal_loan(amount=1000, 
                   nrate=nrate, 
                   grace=5, 
                   dispoints=0, 
                   orgpoints=0,
                   prepmt=None, 
                   balloonpmt=cf.cashflow(const_value=0, nper = 11, spec=(5, 500)))

t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)     0.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(2,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(3,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(4,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(5,)  1000.00   10.00  600.00  100.00  500.00  500.00
(6,)   500.00    8.00  140.00   40.00  100.00  400.00
(7,)   400.00    8.00  132.00   32.00  100.00  300.00
(8,)   300.00    8.00  124.00   24.00  100.00  200.00
(9,)   200.00    8.00  116.00   16.00  100.00  100.00
(10,)  100.00    8.00  108.00    8.00  100.00    0.00

## Ballon loan

En este tipo de créditos el pago se realiza en cuotas fijas. La tasa de interés es fija (no cambia en el tiempo). En el siguiente ejemplo se modela un préstamo de \$ 1000 a un plazo de 10 años y una tasa del 10%.  

In [211]:
cf.fixed_rate_loan(amount=1000, # monto
                nrate=10,    # tasa de interés por periodo
                life=10,     # número de cuotas
                start=None,
                grace=0,
                dispoints=0,
                orgpoints=0,
                prepmt=None,
                balloonpmt=None)


t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)  1000.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  162.75  100.00   62.75  937.25
(2,)   937.25   10.00  162.75   93.73   69.02  868.23
(3,)   868.23   10.00  162.75   86.82   75.92  792.31
(4,)   792.31   10.00  162.75   79.23   83.51  708.80
(5,)   708.80   10.00  162.75   70.88   91.87  616.93
(6,)   616.93   10.00  162.75   61.69  101.05  515.88
(7,)   515.88   10.00  162.75   51.59  111.16  404.72
(8,)   404.72   10.00  162.75   40.47  122.27  282.45
(9,)   282.45   10.00  162.75   28.25  134.50  147.95
(10,)  147.95   10.00  162.75   14.80  147.95    0.00

In [238]:

##
## en éste tipo de crédito es posible considerar pagos adicionales
## a los pagos periodicos con el fin de disminuir el monto periodico
## en el siguiente ejemplo se supone pagos de \$ 200
## en los periodos 5 y 10
##
cf.fixed_rate_loan(amount=1000, # monto
                nrate=10,    # tasa de interés por periodo
                life=10,     # número de cuotas
                start=None,
                grace=0,
                dispoints=0,
                orgpoints=0,
                prepmt=None,
                balloonpmt=cf.cashflow(const_value=0,nper=11,spec=[(5,200),(10,200)]))


t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)  1000.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  129.99  100.00   29.99  970.01
(2,)   970.01   10.00  129.99   97.00   32.98  937.03
(3,)   937.03   10.00  129.99   93.70   36.28  900.75
(4,)   900.75   10.00  129.99   90.07   39.91  860.84
(5,)   860.84   10.00  329.99   86.08  243.90  616.93
(6,)   616.93   10.00  129.99   61.69   68.29  548.64
(7,)   548.64   10.00  129.99   54.86   75.12  473.52
(8,)   473.52   10.00  129.99   47.35   82.63  390.88
(9,)   390.88   10.00  129.99   39.09   90.90  299.99
(10,)  299.99   10.00  329.99   30.00  299.99    0.00

In [239]:
##
## este tipo de créditos tambien admiten el prepago de deuda
## a continuación se modela el caso anterior con un prepago
## de $ 100 en los periodos 3 y 7
##
pmt = cf.cashflow(const_value=0, nper = 11, spec=[(3, 100),(7, 100)])
cf.fixed_rate_loan(amount=1000, # monto
                nrate=10,    # tasa de interés por periodo
                life=10,     # número de cuotas
                start=None,
                grace=0,
                dispoints=0,
                orgpoints=0,
                prepmt=pmt,
                balloonpmt=cf.cashflow(const_value=0,nper=11,spec=[(5,200),(10,200)]))




t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)  1000.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  129.99  100.00   29.99  970.01
(2,)   970.01   10.00  129.99   97.00   32.98  937.03
(3,)   937.03   10.00  229.99   93.70  136.28  800.75
(4,)   800.75   10.00  129.99   80.07   49.91  750.84
(5,)   750.84   10.00  329.99   75.08  254.90  495.93
(6,)   495.93   10.00  129.99   49.59   80.39  415.54
(7,)   415.54   10.00  229.99   41.55  188.43  227.11
(8,)   227.11   10.00  129.99   22.71  107.28  119.83
(9,)   119.83   10.00  129.99   11.98  118.00    1.83
(10,)    1.83   10.00    2.01    0.18    1.83    0.00

In [240]:
##
## un crédito bullet puede modelarse como balloon 
## usando un prepago igual al monto de la deuda 
## 


cf.fixed_rate_loan(amount=1000, # monto
                nrate=10,    # tasa de interés por periodo
                life=10,     # número de cuotas
                start=None,
                grace=0,
                dispoints=0,
                orgpoints=0,
                prepmt=None,
                balloonpmt=cf.cashflow(const_value=0,nper=11,spec=(10,1000)))


t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)  1000.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(2,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(3,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(4,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(5,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(6,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(7,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(8,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(9,)  1000.00   10.00  100.00  100.00    0.00 1000.00
(10,) 1000.00   10.00 1100.00  100.00 1000.00    0.00

## Buydown loans

Este tipo de préstamo es similar a los préstamos de cuota fija (balloon loans) pero con tasa cambiante en el tiempo. No admiten cuotas adicionales para reducir el pago fijo periodico. Cuando hay un cambio en la tasa de interés, el pago periodico total se recalcula para reflejar el cambio de tasa de interés.  

In [243]:
##
## los resultados son iguales a un crédito balloon
##
nrate = cf.nominal_rate(const_value=10, nper=11)
cf.buydown_loan(amount=1000, 
                nrate=nrate, 
                dispoints=0, 
                orgpoints=0, 
                prepmt=None)


t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)  1000.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  162.75  100.00   62.75  937.25
(2,)   937.25   10.00  162.75   93.73   69.02  868.23
(3,)   868.23   10.00  162.75   86.82   75.92  792.31
(4,)   792.31   10.00  162.75   79.23   83.51  708.80
(5,)   708.80   10.00  162.75   70.88   91.87  616.93
(6,)   616.93   10.00  162.75   61.69  101.05  515.88
(7,)   515.88   10.00  162.75   51.59  111.16  404.72
(8,)   404.72   10.00  162.75   40.47  122.27  282.45
(9,)   282.45   10.00  162.75   28.25  134.50  147.95
(10,)  147.95   10.00  162.75   14.80  147.95    0.00

In [244]:
##
## cambios en la tasa de interés
##
nrate = cf.nominal_rate(const_value=10, nper=11,spec=(5,5))
cf.buydown_loan(amount=1000, 
                nrate=nrate, 
                dispoints=0, 
                orgpoints=0, 
                prepmt=None)



t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)  1000.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  162.75  100.00   62.75  937.25
(2,)   937.25   10.00  162.75   93.73   69.02  868.23
(3,)   868.23   10.00  162.75   86.82   75.92  792.31
(4,)   792.31   10.00  162.75   79.23   83.51  708.80
(5,)   708.80    5.00  139.65   35.44  104.21  604.59
(6,)   604.59    5.00  139.65   30.23  109.42  495.18
(7,)   495.18    5.00  139.65   24.76  114.89  380.29
(8,)   380.29    5.00  139.65   19.01  120.63  259.66
(9,)   259.66    5.00  139.65   12.98  126.66  133.00
(10,)  133.00    5.00  139.65    6.65  133.00    0.00

In [245]:
##
## se introducen 2 prepagos por $ 100 cada uno
## en t = 3, 6
##
nrate = cf.nominal_rate(const_value=10, nper=11)
cf.buydown_loan(amount=1000, 
                nrate=nrate, 
                dispoints=0, 
                orgpoints=0, 
                prepmt=cf.cashflow(const_value=0,nper=11,spec=[(3,100),(6,100)]))


t        Beg.    Per.   Total    Int.    Ppal  Ending
         Ppal    Rate     Pmt     Pmt     Pmt    Ppal
-----------------------------------------------------
(0,)  1000.00   10.00    0.00    0.00    0.00 1000.00
(1,)  1000.00   10.00  162.75  100.00   62.75  937.25
(2,)   937.25   10.00  162.75   93.73   69.02  868.23
(3,)   868.23   10.00  262.75   86.82  175.92  692.31
(4,)   692.31   10.00  142.20   69.23   72.97  619.34
(5,)   619.34   10.00  142.20   61.93   80.27  539.07
(6,)   539.07   10.00  242.20   53.91  188.30  350.77
(7,)   350.77   10.00  110.66   35.08   75.58  275.19
(8,)   275.19   10.00  110.66   27.52   83.14  192.05
(9,)   192.05   10.00  110.66   19.21   91.45  100.60
(10,)  100.60   10.00  110.66   10.06  100.60    0.00

---

**Ejercicio.--** Verifique manualmente todos los cálculos realizados para los diferentes tipos de créditos.

---

[Contenido](#Contenido)